In [ ]:
import pandas as pd
import utils.fetcher_utils as fetcher
import utils.preprocess_util as preproc

import numpy as np
import matplotlib.pyplot as plt
import locale


In [ ]:
# set pandas options
pd.set_option('display.max_columns', None)


In [ ]:
# fetch the data 
df = fetcher.aquireIMDbDataFrame()
df.head()

In [ ]:
with pd.option_context('display.float_format', '{:.2f}'.format):
    print(df['gross'].describe())

In [ ]:
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
print(f"Maximum Gross: {locale.currency(df['gross'].max(), grouping=True)}")
print(f"Minimum Gross: {locale.currency(df['gross'].min(), grouping=True)}")

# This looks weird, let's investigate

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Boxplot
axes[0].boxplot(df['gross'].dropna(), vert=False, patch_artist=True)
axes[0].set_title("Boxplot of Gross Values")
axes[0].set_xlabel("Gross")

# Histogram
axes[1].hist(df['gross'].dropna(), bins=10, edgecolor='k', alpha=0.7)
axes[1].set_title("Histogram of Gross Values")
axes[1].set_xlabel("Gross")
axes[1].set_ylabel("Frequency")

plt.tight_layout()
plt.show()

In [ ]:
gross_cleaned = df['gross'].dropna()

# Calculate Q1 (25th percentile) and Q3 (75th percentile)
Q1 = gross_cleaned.quantile(0.25)
Q3 = gross_cleaned.quantile(0.75)

# Compute IQR
IQR = Q3 - Q1

# Calculate outlier boundaries
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
outliers = gross_cleaned[(gross_cleaned < lower_bound) | (gross_cleaned > upper_bound)]

print(f"Q1: {Q1}, Q3: {Q3}, IQR: {IQR}")
print(f"Lower Bound: {lower_bound}, Upper Bound: {upper_bound}")
print("Outliers:")
print(outliers)

In [ ]:
df['gross_capped'] = df['gross'].clip(lower=lower_bound, upper=upper_bound)
df['gross_capped'].count()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Boxplot
axes[0].boxplot(df['gross_capped'].dropna(), vert=False, patch_artist=True)
axes[0].set_title("Boxplot of Gross Values")
axes[0].set_xlabel("Gross")

# Histogram
axes[1].hist(df['gross'].dropna(), bins=10, edgecolor='k', alpha=0.7)
axes[1].set_title("Histogram of Gross Values")
axes[1].set_xlabel("Gross")
axes[1].set_ylabel("Frequency")

plt.tight_layout()
plt.show()